Import Libraries and Load Dataset

In [10]:
class SaveEveryNEpochs(tf.keras.callbacks.Callback):
    def __init__(self, n, filepath):
        super().__init__()
        self.n = n
        self.filepath = filepath

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.n == 0:
            path = self.filepath.format(epoch=epoch + 1)
            self.model.save_weights(path)
            print(f"\nSaved weights at: {path}")

In [11]:
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

def get_latest_checkpoint():
    # Look for files like epoch_0010.weights.h5
    pattern = os.path.join(checkpoint_dir, "epoch_*.weights.h5")
    files = glob.glob(pattern)
    if not files:
        return None, 0  # no checkpoint found

    # Extract the epoch number using regex
    def extract_epoch(path):
        match = re.search(r"epoch_(\d+)\.weights\.h5", path)
        return int(match.group(1)) if match else -1

    latest_file = max(files, key=extract_epoch)
    latest_epoch = extract_epoch(latest_file)
    return latest_file, latest_epoch

In [12]:
checkpoint_cb = SaveEveryNEpochs(
    n=1,
    filepath="checkpoints/epoch_{epoch:04d}.weights.h5"
)

In [13]:
latest_ckpt, last_epoch = get_latest_checkpoint()
print("Latest checkpoint:", latest_ckpt)
print("Last epoch number:", last_epoch)

Latest checkpoint: checkpoints/epoch_0300.weights.h5
Last epoch number: 300


In [ ]:
# model = PUT THE CODE HERE THAT DEFINES YOUR MODEL



# Load weights if checkpoint exists
if latest_ckpt is not None:
    print(f"Loading weights from {latest_ckpt}")
    model.load_weights(latest_ckpt)
else:
    print("No checkpoint found; starting from scratch.")

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=masked_loss,            # custom loss that ignores padding
    metrics=[masked_accuracy],   # custom accuracy that ignores padding
)

model.summary()

/opt/anaconda3/envs/tf216metal/lib/python3.11/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'positional_embedding' (of type SinusoidalPositionalEncoding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Loading weights from checkpoints/epoch_0300.weights.h5


Model: "DadLLM"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ token_ids           │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_embedding     │ (None, 64, 768)   │ 30,720,000 │ token_ids[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, 64, 768)   │          0 │ token_embedding[… │
│ (SinusoidalPositio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mha_1               │ (None, 64, 768)   │  2,362,368 │ positional_embed… │
│ (MultiHeadAttentio… │                   │            │ positional_embed… │
│                     │                   │            │ positional_embed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 64, 768)   │          0 │ positional_embed… │
│                     │                   │            │ mha_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ln_attn_1           │ (None, 64, 768)   │      1,536 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn1_1 (Dense)      │ (None, 64, 3072)  │  2,362,368 │ ln_attn_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn2_1 (Dense)      │ (None, 64, 768)   │  2,360,064 │ ffn1_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 64, 768)   │          0 │ ln_attn_1[0][0],  │
│                     │                   │            │ ffn2_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ln_ffn_1            │ (None, 64, 768)   │      1,536 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mha_2               │ (None, 64, 768)   │  2,362,368 │ ln_ffn_1[0][0],   │
│ (MultiHeadAttentio… │                   │            │ ln_ffn_1[0][0],   │
│                     │                   │            │ ln_ffn_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 64, 768)   │          0 │ ln_ffn_1[0][0],   │
│                     │                   │            │ mha_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ln_attn_2           │ (None, 64, 768)   │      1,536 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn1_2 (Dense)      │ (None, 64, 3072)  │  2,362,368 │ ln_attn_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn2_2 (Dense)      │ (None, 64, 768)   │  2,360,064 │ ffn1_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 64, 768)   │          0 │ ln_attn_2[0][0],  │
│                     │                   │            │ ffn2_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ln_ffn_2            │ (None, 64, 768)   │      1,536 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 146,534,464 (558.98 MB)

 Trainable params: 146,534,464 (558.98 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    # put stuff here about your test and validation data
    epochs=50,                 # total desired epochs
    # If your last weights saved was epoch 25, and you have epochs=50
    # Then retraining will only do 25 new epochs.
    initial_epoch=last_epoch,  # resume here
    callbacks=[checkpoint_cb],
)

Epoch 251/300


W0000 00:00:1765330762.814225    6463 assert_op.cc:38] Ignoring Assert operator compile_loss/masked_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


3704/3705 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.3139 - masked_accuracy: 0.9152

W0000 00:00:1765331061.684052    6463 assert_op.cc:38] Ignoring Assert operator compile_loss/masked_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


3705/3705 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.3139 - masked_accuracy: 0.9152

W0000 00:00:1765331076.880793    6466 assert_op.cc:38] Ignoring Assert operator compile_loss/masked_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1765331086.824801    6466 assert_op.cc:38] Ignoring Assert operator compile_loss/masked_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert



Saved weights at: checkpoints/epoch_0251.weights.h5
3705/3705 ━━━━━━━━━━━━━━━━━━━━ 339s 84ms/step - loss: 0.2985 - masked_accuracy: 0.9201 - val_loss: 0.8266 - val_masked_accuracy: 0.8691
Epoch 252/300
3704/3705 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.3131 - masked_accuracy: 0.9148
Saved weights at: checkpoints/epoch_0252.weights.h5
3705/3705 ━━━━━━━━━━━━━━━━━━━━ 271s 73ms/step - loss: 0.2968 - masked_accuracy: 0.9198 - val_loss: 0.8177 - val_masked_accuracy: 0.8714
Epoch 253/300
3704/3705 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.3114 - masked_accuracy: 0.9151
Saved weights at: checkpoints/epoch_0253.weights.h5
3705/3705 ━━━━━━━━━━━━━━━━━━━━ 271s 73ms/step - loss: 0.2954 - masked_accuracy: 0.9201 - val_loss: 0.8261 - val_masked_accuracy: 0.8685
Epoch 254/300
3704/3705 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.3116 - masked_accuracy: 0.9149
Saved weights at: checkpoints/epoch_0254.weights.h5
3705/3705 ━━━━━━━━━━━━━━━━━━━━ 271s 73ms/step - loss: 0.2955 - masked_accuracy: 0.9200